# Main Code

In [1]:
!pip install accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found exist

In [26]:
!pip install -U transformers
# !huggingface-cli login

In [4]:
from huggingface_hub import login

# Paste your token directly here as a string (yes, it's visible – be careful not to share this notebook)

In [5]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers.git --upgrade

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-nwipe79q
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-nwipe79q
  Resolved https://github.com/huggingface/transformers.git to commit 31f8a0fe8a7e2db1ee30bf32ed5976cd11f3283c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.0.dev0-py3-none-any.whl size=11365971 sha256=87ed7fb2322c86935fb8c1fb8b7c0b0ece9e42a2dcc35ef44c09cb311238401f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ulff_f59/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Replace with any LLaMA model you have access to
model_id = "meta-llama/Llama-2-7b-chat-hf"  # or Llama-3-8b-instruct if available

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True  # Save memory
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0.2,
    repetition_penalty=1.1
)

2025-05-24 11:23:27.568507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748085807.746705      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748085807.794720      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
chat_template = """
You are an AI assistant for an insurance company. Be helpful and polite.

Client says: {query}

Your reply:
"""

extract_template = """
Extract the following information from the client message:

- Full Name
- Date of Birth
- Insurance Type
- Description of Issue
- Policy Number

Only respond in JSON format:
{{
  "name": "...",
  "dob": "...",
  "insurance_type": "...",
  "issue_description": "...",
  "policy_number": "..."
}}

Client message: {query}
"""

# 🧠 Chatbot logic
def insurance_chat(query):
    # Compose prompts
    chat_prompt = chat_template.format(query=query)
    extract_prompt = extract_template.format(query=query)

    # Generate response
    response = pipe(chat_prompt)[0]['generated_text'].split("Your reply:")[-1].strip()
    extracted = pipe(extract_prompt)[0]['generated_text'].split("```json")[-1].strip()

    # Show results
    print("🤖 Assistant:\n", response)
    print("\n📄 Extracted Info:\n", extracted)

In [11]:
chat_template = """
You are an AI assistant for an insurance company. Be helpful and polite.

Client says: {query}

Your reply:
"""

extract_template = """
Extract the following information from the client message:

- Full Name
- Date of Birth
- Insurance Type
- Description of Issue
- Policy Number

Respond only with a valid JSON object:
{{
  "name": "...",
  "dob": "...",
  "insurance_type": "...",
  "issue_description": "...",
  "policy_number": "..."
}}

Client message: {query}
"""

import json

def insurance_chat(query):
    # Compose prompts
    chat_prompt = chat_template.format(query=query)
    extract_prompt = extract_template.format(query=query)

    # Generate chat response
    response_raw = pipe(chat_prompt)[0]['generated_text']
    response = response_raw.split("Your reply:")[-1].strip()

    # Generate structured extraction
    extracted_raw = pipe(extract_prompt)[0]['generated_text'].strip()

    # Try to parse the JSON output
    try:
        extracted_json = json.loads(extracted_raw)
    except json.JSONDecodeError:
        print("⚠️ JSON parsing failed. Raw output:\n", extracted_raw)
        extracted_json = None

    # Show results
    print("🤖 Assistant:\n", response)
    print("\n📄 Extracted Info:\n", extracted_json if extracted_json else extracted_raw)


In [12]:
insurance_chat("Hi, my name is Alice Chen. My car has broke down on Langston Street a few hours ago can you help?")

⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message:

- Full Name
- Date of Birth
- Insurance Type
- Description of Issue
- Policy Number

Respond only with a valid JSON object:
{
  "name": "...",
  "dob": "...",
  "insurance_type": "...",
  "issue_description": "...",
  "policy_number": "..."
}

Client message: Hi, my name is Alice Chen. My car has broke down on Langston Street a few hours ago can you help?

Please provide your insurance details and policy number so we can process your claim.
🤖 Assistant:
 Of course, Ms. Chen! I'm so sorry to hear that your car broke down. Can you please provide me with more details about the situation? Where exactly did it happen and what time did it occur? Additionally, do you have any idea what might have caused the breakdown? Any information you can provide will help us assist you better.

📄 Extracted Info:
 Extract the following information from the client message:

- Full Name
- Date of Birth
- Insuranc

In [13]:
chat_template = """
Είσαι ένας βοηθητικός και ευγενικός ψηφιακός βοηθός για μια ασφαλιστική εταιρεία.

Ο πελάτης λέει: {query}

Η απάντησή σου:
"""

extract_template = """
Εξήγαγε τις εξής πληροφορίες από το μήνυμα του πελάτη:

- Ονοματεπώνυμο
- Ημερομηνία γέννησης
- Τύπος ασφάλισης
- Περιγραφή προβλήματος
- Αριθμός συμβολαίου

Απάντησε μόνο με ένα έγκυρο αντικείμενο JSON:
{{
  "name": "...",
  "dob": "...",
  "insurance_type": "...",
  "issue_description": "...",
  "policy_number": "..."
}}

Μήνυμα πελάτη: {query}
"""

In [14]:
insurance_chat("Καλησπέρα, ονομάζομαι Ιωάννης Παπαδόπουλος, γεννήθηκα στις 15 Μαρτίου 1980. Το αμαξι μου χαλασε στην οδο Παπαδακη 68 μπορειτε να στειλετε την βοήθεια δρόμου")

⚠️ JSON parsing failed. Raw output:
 Εξήγαγε τις εξής πληροφορίες από το μήνυμα του πελάτη:

- Ονοματεπώνυμο
- Ημερομηνία γέννησης
- Τύπος ασφάλισης
- Περιγραφή προβλήματος
- Αριθμός συμβολαίου

Απάντησε μόνο με ένα έγκυρο αντικείμενο JSON:
{
  "name": "...",
  "dob": "...",
  "insurance_type": "...",
  "issue_description": "...",
  "policy_number": "..."
}

Μήνυμα πελάτη: Καλησπέρα, ονομάζομαι Ιωάννης Παπαδόπουλος, γεννήθηκα στις 15 Μαρτίου 1980. Το αμαξι μου χαλασε στην οδο Παπαδακη 68 μπορειτε να στειλετε την βοήθεια δρόμου

Σύμβουλος: Σας προσεκτάμε να προσθέσθει μια βοήθεια δρόμου στην οδό Παπαδακη 68, όπως η βοήθεια δρόμου στην οδό Δελφίνη 74.
🤖 Assistant:
 Είσαι ένας βοηθητικός και ευγενικός ψηφιακός βοηθός για μια ασφαλιστική εταιρεία.

Ο πελάτης λέει: Καλησπέρα, ονομάζομαι Ιωάννης Παπαδόπουλος, γεννήθηκα στις 15 Μαρτίου 1980. Το αμαξι μου χαλασε στην οδο Παπαδακη 68 μπορειτε να στειλετε την βοήθεια δρόμου

Η απάντησή σου:

Ωρα παρακληθώ, ποτέ θα περνά σας ένα βοηθή και ευγενικ

In [8]:
!pip install -U langchain langchain-community --quiet
# ✅ 1. Install necessary packages (no need for langchain-together anymore)
!pip install -U transformers accelerate langchain huggingface_hub sentencepiece --quiet

# ✅ 2. Imports
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_core.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00


In [10]:
# ✅ 3. Choose DeepSeek model (no gated access)
model_id = "deepseek-ai/DeepSeek-R1"  # You can change to deepseek-coder-* if desired

# ✅ 4. Load tokenizer and model (auto-detects GPU if available)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# ✅ 5. Build a pipeline for inference
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0.0,
    repetition_penalty=1.1
)

# ✅ 6. Wrap in LangChain LLM interface
llm = HuggingFacePipeline(pipeline=pipe)

# ✅ 7. Create prompt template
template = """You are an AI assistant for an insurance company. Extract relevant information and respond accurately.
Question: {query}"""
# ✅ Install latest dependencies
!pip install -U transformers accelerate sentencepiece bitsandbytes langchain langchain-community --quiet

# ✅ Imports
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate

import torch

# ✅ DeepSeek-R1 model
model_id = "deepseek-ai/DeepSeek-R1"

# ✅ Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# ✅ Hugging Face pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0.0,
    repetition_penalty=1.1
)

# ✅ LangChain wrapper
llm = HuggingFacePipeline(pipeline=pipe)

# ✅ Prompt template
template = """You are an AI assistant for an insurance company. Extract relevant information and respond accurately.
Question: {query}"""
prompt = PromptTemplate.from_template(template)

# ✅ Chain
chain = prompt | llm

# ✅ Test
response = chain.invoke({"query": "Hi, my name is John. I had a car accident in New York on May 10th."})
print(response)


tokenizer_config.json:   0%|          | 0.00/3.59k [00:00<?, ?B/s]

RuntimeError: Failed to import transformers.models.llama.tokenization_llama_fast because of the following error (look up to see its traceback):
cannot import name 'requires' from 'transformers.utils.import_utils' (/usr/local/lib/python3.11/dist-packages/transformers/utils/import_utils.py)

In [13]:
# 🚫 Uninstall broken versions
!pip uninstall -y transformers
!pip uninstall -y tokenizers
!pip uninstall -y huggingface_hub
!pip uninstall -y accelerate

# ✅ Reinstall cleanly with versions compatible with DeepSeek
!pip install -U "transformers==4.40.1" "accelerate>=0.28.0" sentencepiece bitsandbytes --quiet

Found existing installation: transformers 4.52.3
Uninstalling transformers-4.52.3:
  Successfully uninstalled transformers-4.52.3
Found existing installation: tokenizers 0.21.1
Uninstalling tokenizers-0.21.1:
  Successfully uninstalled tokenizers-0.21.1
Found existing installation: huggingface-hub 0.32.0
Uninstalling huggingface-hub-0.32.0:
  Successfully uninstalled huggingface-hub-0.32.0
Found existing installation: accelerate 1.7.0
Uninstalling accelerate-1.7.0:
  Successfully uninstalled accelerate-1.7.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 40.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 47.2 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0,

In [15]:
# Uninstall the current one
!pip uninstall -y transformers

# Install the latest bleeding-edge transformers from GitHub (with deepseek_v3 support)
!pip install git+https://github.com/huggingface/transformers.git --upgrade --quiet

Found existing installation: transformers 4.40.1
Uninstalling transformers-4.40.1:
  Successfully uninstalled transformers-4.40.1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.2 MB/s eta 0:00:0000:01


In [16]:
import transformers
print(transformers.__version__)

4.51.3


In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
import torch

model_id = "deepseek-ai/DeepSeek-R1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0.0,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=pipe)

template = """You are an AI assistant for an insurance company. Extract relevant information and respond accurately.
Question: {query}"""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

response = chain.invoke({"query": "Hi, my name is John. I had a car accident in New York on May 10th."})
print(response)

RuntimeError: Failed to import transformers.models.deepseek_v3.configuration_deepseek_v3 because of the following error (look up to see its traceback):
No module named 'transformers.models.deepseek_v3.configuration_deepseek_v3'

In [20]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers.git --upgrade

Found existing installation: transformers 4.53.0.dev0
Uninstalling transformers-4.53.0.dev0:
  Successfully uninstalled transformers-4.53.0.dev0
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-2n847kjt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-2n847kjt
  Resolved https://github.com/huggingface/transformers.git to commit 31f8a0fe8a7e2db1ee30bf32ed5976cd11f3283c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.0.dev0-py3-none-any.whl size=11365971 sha256=c870c63c44139f0c56ece15d5d7e97685cfd5a10af64136423fc87702c571432
  Stored in directory: /tmp/pip-ephem-wheel-cache-lfz846jg/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers


In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
import torch

model_id = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0.0,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=pipe)

template = """You are an AI assistant for an insurance company. Extract relevant information and respond accurately.
Question: {query}"""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

response = chain.invoke({"query": "Hi, my name is John. I had a car accident in New York on May 10th."})
print(response)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

RuntimeError: Failed to import transformers.models.cohere2.configuration_cohere2 because of the following error (look up to see its traceback):
cannot import name 'layer_type_validation' from 'transformers.configuration_utils' (/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py)

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Replace with any LLaMA model you have access to
model_id = "meta-llama/Llama-3.1-8B-Instruct"  # or Llama-3-8b-instruct if available

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True  # Save memory
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0.2,
    repetition_penalty=1.1
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


In [26]:
chat_template = """
You are an AI assistant for the insurance company named Greek Insurance Company.
You will be contacted by different types of people when they have a problem with their car.
Your first role from the message you receive from the client it to determine in what category they belong into.
The four character categories are:
1) Clients that need accident care aka have either been in a big accident, head-on collision or any type of accident that involves multiple vehicles.
2) Clients that need road assistance aka the car has shown any kind of malfunction and does not move normall so the client cannot go to their intended destination
3) Clients that have not provided enough information to determine whether they are one of the top two categories so we ask a question for clarification
4) Clients that by their way of talking or mocking remarks or lack of any information after a couple of questions we determine are not serious callers and are doing a phone prank.
In the last case your response should be dismissive of our services.
In the third case indirectly try to ask them what exactly happened so we get that information eventually.
In the first two cases you should ask for as many of pieces of information listed below we can get.
The information we want to hear from the client include
- Full Name
- Category of Message (as mentioned above)
- Location of Accident/Malfunction
- What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
call accident care to note the accident who was at fault etc.)
- In what location should they end up (service center, home etc.)
- The exact location of the above location point
If you dont get one of the above info start by understanding the client and offering your apologies for what happened
and then try to get as many of the other pieces of information without being overbearing
Always be respectful and nice to the client.
Be very clear with your answers to the client.
The ONLY thing you should return is the message towards the client NOTHING MORE


Client says: {query}

Your reply:
"""

extract_template = """
Extract the following information from the client message:

- Full Name
- Category of Message (as mentioned above)
- Location of Accident/Malfunction
- What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
call accident care to note the accident who was at fault etc.)
- In what location should they end up (service center, home etc.)
- The exact location of the above location point

If one of the pieces of info is not available just put the word None on it and return the JSON normally

Respond only with a valid JSON object:
{{
  "name": "...",
  "dob": "...",
  "insurance_type": "...",
  "issue_description": "...",
  "policy_number": "..."
}}

Client message: {query}
"""

import json

def insurance_chat(query):
    # Compose prompts
    chat_prompt = chat_template.format(query=query)
    extract_prompt = extract_template.format(query=query)
    # Generate chat response
    response_raw = pipe(chat_prompt)[0]['generated_text']
    response = response_raw.split("Your reply:")[-1].strip()

    # Generate structured extraction
    extracted_raw = pipe(extract_prompt)[0]['generated_text'].strip()

    # Try to parse the JSON output
    try:
        extracted_json = json.loads(extracted_raw)
    except json.JSONDecodeError:
        print("⚠️ JSON parsing failed. Raw output:\n", extracted_raw)
        extracted_json = None

    # Show results
    print("🤖 Assistant:\n", response)
    print("\n📄 Extracted Info:\n", extracted_json if extracted_json else extracted_raw)


In [27]:
insurance_chat("Hi, my name is Alice Chen. My car has broke down on Langston Street a few hours ago can you help?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message:

- Full Name
- Category of Message (as mentioned above)
- Location of Accident/Malfunction
- What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
call accident care to note the accident who was at fault etc.)
- In what location should they end up (service center, home etc.)
- The exact location of the above location point

If one of the pieces of info is not available just put the word None on it and return the JSON normally

Respond only with a valid JSON object:
{
  "name": "...",
  "dob": "...",
  "insurance_type": "...",
  "issue_description": "...",
  "policy_number": "..."
}

Client message: Hi, my name is Alice Chen. My car has broke down on Langston Street a few hours ago can you help?
I would like to request roadside assistance to have my car fixed as soon as possible.
My car's license plate number is ABC123.

Response JSON:
{
  

In [29]:
insurance_chat("Hello my name is John Doe and I have been in an accident. There was a car coming the other way and we just collided head-on there are no serious injuries but we need you to come quickly!")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message:

- Full Name
- Category of Message (as mentioned above)
- Location of Accident/Malfunction
- What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
call accident care to note the accident who was at fault etc.)
- In what location should they end up (service center, home etc.)
- The exact location of the above location point

If one of the pieces of info is not available just put the word None on it and return the JSON normally

Respond only with a valid JSON object:
{
  "name": "...",
  "dob": "...",
  "insurance_type": "...",
  "issue_description": "...",
  "policy_number": "..."
}

Client message: Hello my name is John Doe and I have been in an accident. There was a car coming the other way and we just collided head-on there are no serious injuries but we need you to come quickly!
Location of accident: 123 Main St, Anytown USA
Category of

In [31]:
# Consider adding these to your pipeline:

# Better error handling for JSON extraction
def safe_json_extraction(text):
    start = text.find('{')
    end = text.rfind('}') + 1
    if start == -1 or end == 0:
        return None
    try:
        return json.loads(text[start:end])
    except:
        return None

# Conversation summary for long chats
def summarize_conversation(history):
    summary_prompt = f"""Summarize this insurance conversation:
    {" ".join(history)}
    
    Focus on:
    - Client's main issue
    - Information gathered
    - Next steps
    
    Summary:"""
    return pipe(summary_prompt)[0]['generated_text']

In [53]:
from collections import defaultdict

# Add this at the top of your script
conversation_histories = defaultdict(list)

def update_chat_template(history):
    system_prompt = """
                You are an AI assistant for the insurance company named Greek Insurance Company.
                You will be contacted by different types of people when they have a problem with their car.
                Your first role from the message you receive from the client it to determine in what category they belong into.
                The four character categories are:
                1) Clients that need accident care aka have either been in a big accident, head-on collision or any type of accident that involves multiple vehicles.
                2) Clients that need road assistance aka the car has shown any kind of malfunction and does not move normall so the client cannot go to their intended destination
                3) Clients that have not provided enough information to determine whether they are one of the top two categories so we ask a question for clarification
                4) Clients that by their way of talking or mocking remarks or lack of any information after a couple of questions we determine are not serious callers and are doing a phone prank.
                In the last case your response should be dismissive of our services.
                In the third case indirectly try to ask them what exactly happened so we get that information eventually.
                In the first two cases you should ask for as many of pieces of information listed below we can get.
                The information we want to hear from the client include
                - Full Name
                - Category of Message (as mentioned above)
                - Location of Accident/Malfunction
                - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
                call accident care to note the accident who was at fault etc.)
                - In what location should they end up (service center, home etc.) 
                - The exact location of the above location point
                If you dont get one of the above info start by understanding the client and offering your apologies for what happened
                and then try to get as many of the other pieces of information without being overbearing
                Always be respectful and nice to the client.
                Be very clear with your answers to the client.
                The ONLY thing you should return is the message towards the client NOTHING MORE
                Don't repeat to the client the info they gave you in previous message
                If you get conflicting information ask for clarification on what is the correct info, precision is important
                Also don't over ask the client if you have all information available and some more it is ok to stop asking
                Always care about their safety
                Try to keep the conversations short and precise and to the point most conversations should last up to 7 messages unless you don't have yet some of the info needed
                Try to diagnose the problem if they talk about why their car broke down or stopped, use your knowledge about what can cause certain accidents
                
                
                Client says: {query}
                
                Your reply:
    """

    messages = [{"role": "system", "content": system_prompt}]
    for i, (user_msg, assistant_msg) in enumerate(zip(history[::2], history[1::2])):
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})
    if len(history) % 2 == 1:
        messages.append({"role": "user", "content": history[-1]})
    
    return tokenizer.apply_chat_template(messages, tokenize=False)

def insurance_chat(query, user_id="default"):
    # Get conversation history
    history = conversation_histories[user_id]
    history.append(query)
    
    # Generate prompt with history
    prompt = update_chat_template(history)
    
    # Generate response
    response_raw = pipe(prompt)[0]['generated_text']
    
    # Extract just the new response
    response = response_raw.split(history[-1])[-1].strip()
    
    # Update history with assistant response
    history.append(response)
    
    # Generate structured extraction (modified to use full conversation)
    extract_prompt = f"""
        Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - In what location should they end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {{
          "name": "...",
          "dob": "...",
          "insurance_type": "...",
          "issue_description": "...",
          "policy_number": "..."
        }}
    
        Client message: {query}
        The history is the following: {" ".join(history)}"""
    
    extracted_raw = pipe(extract_prompt)[0]['generated_text'].strip()
    
    try:
        extracted_json = json.loads(extracted_raw)
    except json.JSONDecodeError:
        print("⚠️ JSON parsing failed. Raw output:\n", extracted_raw)
        extracted_json = None

    print("🤖 Assistant:\n", response)
    print("\n📄 Extracted Info:\n", extracted_json if extracted_json else extracted_raw)
    
    return response, extracted_json

In [54]:
insurance_chat("Hi, my name is Alice Chen. My car has broke down on Langston Street a few hours ago can you help?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - In what location should they end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "dob": "...",
          "insurance_type": "...",
          "issue_description": "...",
          "policy_number": "..."
        }
    
        Client message: Hi, my name is Alice Chen. My car has broke down on Langston Street a few hou

("<|eot_id|>assistant\n\nHello Alice, I'm glad you reached out to us. Sorry to hear that your car broke down. Can you tell me where on Langston Street you're located, please? Is it near any landmarks or intersections that might help me narrow down the location?\n\nAlso, would you like Road Assistance to send someone to help you fix your car, or would you prefer to have it towed to a service center?",
 None)

In [55]:
insurance_chat("Yes I am currectly on Georgian Street 25 and can you please send road assistance immediately. I have to take the car to the service center!! I don't know what happened exactly but it seemed to put our some kind of screetching noise before stopping")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - In what location should they end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "dob": "...",
          "insurance_type": "...",
          "issue_description": "...",
          "policy_number": "..."
        }
    
        Client message: Yes I am currectly on Georgian Street 25 and can you please send road assista

("<|eot_id|>assistant\n\nAlice, thank you for providing the new location. I've noted that you're currently on Georgian Street 25.\n\nI'd be happy to send Road Assistance to help you. They'll arrive shortly to assist you. As for the issue with your car making a screeching noise before stopping, I'm going to take a guess that it might be related to a potential brake problem. However, our technician will assess this further once they arrive.\n\nTo confirm, you'd like to have your car taken to our Service Center, correct? And would you like it to be dropped off at the main entrance or would you prefer a specific bay number?",
 None)

In [56]:
insurance_chat("I don't have a problem with the bay number and just teh closest service center is fine")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - In what location should they end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "dob": "...",
          "insurance_type": "...",
          "issue_description": "...",
          "policy_number": "..."
        }
    
        Client message: I don't have a problem with the bay number and just teh closest service cente

("<|eot_id|>assistant\n\nAlice, I've dispatched Road Assistance to your location on Georgian Street 25. They'll arrive shortly to assist you.\n\nAs for the Service Center, I've noted that you'd like to drop off your vehicle at the nearest one. Our team will meet you there and take care of your car's assessment.\n\nBefore they arrive, I just want to confirm: Are you safe and comfortable where you are now, or would you like me to arrange for an escort to accompany you to the Service Center?",
 None)

In [57]:
insurance_chat("No no I am fine thank you for the help!")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - In what location should they end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "dob": "...",
          "insurance_type": "...",
          "issue_description": "...",
          "policy_number": "..."
        }
    
        Client message: No no I am fine thank you for the help!
        The history is the following:

("<|eot_id|>assistant\n\nAlice, great, I'm glad to hear that you're okay. Our team from Road Assistance will arrive soon to help you with your car. They'll take good care of you and your vehicle.\n\nIn the meantime, I've sent you a confirmation text with the details of the assistance, including the estimated arrival time and the Service Center information. Please look out for it.\n\nIs there anything else I can assist you with before they arrive?",
 None)

In [58]:
insurance_chat("That's all")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - In what location should they end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "dob": "...",
          "insurance_type": "...",
          "issue_description": "...",
          "policy_number": "..."
        }
    
        Client message: That's all
        The history is the following: Hi, my name is Alice Chen. M

("<|eot_id|>assistant\n\nAlice, you're all set. Help is on the way. Have a safe wait and I'll pass along any additional information to our team. Thank you for reaching out to us, and we hope to see you soon at the Service Center.",
 None)

# Generate more prompts

In [ ]:
import random
import json
from faker import Faker
from transformers import pipeline
from typing import List, Dict

# Initialize tools
fake = Faker()
generator = pipeline("text-generation", model="meta-llama/Meta-Llama-3-8B-Instruct")

# Load your real conversations (replace with your actual data)
real_conversations = [
    {
        "messages": [
            {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
            {"role": "user", "content": "I just crashed my car near Syntagma Square!"},
            {"role": "assistant", "content": "I'm sorry to hear about the accident. May I have your full name?"},
            {"role": "user", "content": "It's Dimitris Ioannou"},
            {"role": "assistant", "content": "Thank you Mr. Ioannou. Were there any other vehicles involved?"}
        ]
    },
    # Add your other 9 real conversations here
]

# Define augmentation parameters
LOCATIONS = ["Syntagma Square", "Thessaloniki Center", "Patras Harbor", 
             "Heraklion Port", "Larissa Train Station", "Volos Waterfront"]
NAMES = [fake.name() for _ in range(50)]  # Generate 50 Greek names
CAR_TYPES = ["Toyota", "Honda", "BMW", "Mercedes", "Fiat", "Peugeot"]
ISSUES = ["engine failure", "flat tire", "battery dead", "accident", "overheating"]

def augment_conversation(original: List[Dict]) -> List[Dict]:
    """Create variations of a real conversation"""
    augmented = []
    for message in original:
        if message["role"] == "user":
            # Augment user messages
            content = message["content"]
            content = content.replace("Syntagma Square", random.choice(LOCATIONS))
            content = content.replace("Dimitris Ioannou", random.choice(NAMES))
            
            # Add random car details
            if random.random() > 0.7:
                car = random.choice(CAR_TYPES)
                content = f"My {car} {random.choice(['had an issue', 'broke down', 'was damaged'])}. {content}"
                
            augmented.append({"role": "user", "content": content})
        else:
            # Keep assistant messages mostly unchanged
            augmented.append(message)
    return augmented

def generate_synthetic_conversation() -> List[Dict]:
    """Generate completely new synthetic conversation"""
    issue = random.choice(ISSUES)
    name = random.choice(NAMES)
    location = random.choice(LOCATIONS)
    
    if "accident" in issue:
        user_msg = f"I just had a car accident near {location}! This is {name}."
        assistant_msgs = [
            f"I'm sorry to hear about the accident Mr. {name.split()[-1]}. Were there any injuries?",
            "Could you describe what happened?",
            f"We'll dispatch a team to {location}. Is your car drivable?"
        ]
    else:
        user_msg = f"My car has {issue} at {location}. I'm {name}."
        assistant_msgs = [
            f"I understand you need road assistance Mr. {name.split()[-1]}.",
            f"Where exactly at {location} are you located?",
            "Is your car safe to drive or do you need a tow truck?"
        ]
    
    # Build conversation
    messages = [
        {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
        {"role": "user", "content": user_msg}
    ]
    
    # Add 2-4 assistant responses
    for i in range(random.randint(2, 4)):
        messages.append({"role": "assistant", "content": assistant_msgs[i % len(assistant_msgs)]})
        if i < 3:  # Add user follow-ups
            follow_up = generate_user_followup(issue, i)
            messages.append({"role": "user", "content": follow_up})
    
    return messages

def generate_user_followup(issue: str, turn: int) -> str:
    """Generate realistic user follow-up messages"""
    if turn == 0:
        return random.choice([
            "Yes, please send help quickly!",
            "No injuries but the car is badly damaged",
            "I'm not sure, it just stopped working"
        ])
    elif turn == 1:
        if "accident" in issue:
            return random.choice([
                "It was a rear-end collision",
                "Another car hit me from the side",
                "I lost control on the wet road"
            ])
        else:
            return random.choice([
                "Near the main intersection",
                "In the parking lot",
                "On the highway shoulder"
            ])
    else:
        return random.choice([
            "Thank you for your help",
            "How long will assistance take?",
            "Should I call the police?"
        ])

# Generate synthetic dataset
synthetic_data = []

# 1. Augment real conversations (5 variations per real conversation)
for conv in real_conversations:
    for _ in range(5):
        synthetic_data.append(augment_conversation(conv))

# 2. Generate completely new conversations (90 synthetic examples)
for _ in range(90):
    synthetic_data.append(generate_synthetic_conversation())

# Save the dataset
with open("synthetic_insurance_conversations.json", "w") as f:
    json.dump(synthetic_data, f, indent=2)

print(f"Generated {len(synthetic_data)} synthetic conversations (originally had {len(real_conversations)})")

In [ ]:
from datasets import load_dataset

# Load your synthetic data
dataset = load_dataset("json", data_files="synthetic_insurance_conversations.json")

# Prepare for training (same as before)
def format_conversation(example):
    example["text"] = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False
    )
    return example

formatted_dataset = dataset.map(format_conversation)

In [ ]:
def refine_with_llm(conversation):
    prompt = f"""Improve this insurance conversation to sound more natural:
    {conversation}
    
    Improved version:"""
    refined = generator(prompt, max_length=1000)[0]['generated_text']
    return refined.split("Improved version:")[-1].strip()

In [ ]:
# Count examples per category
category_counts = {
    "accident": 0,
    "roadside": 0,
    "pranks": 0
}

# Generate until we have enough of each
while min(category_counts.values()) < 20:
    new_conv = generate_synthetic_conversation()
    category = classify_conversation(new_conv)  # Implement this
    if category_counts[category] < 20:
        synthetic_data.append(new_conv)
        category_counts[category] += 1

# More Code

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load a lightweight classifier model (e.g., distilled BERT)
model_name = "distilbert-base-uncased"  # or "microsoft/Phi-3-mini-4k-instruct" for better results
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a classification layer on top
class ConversationClassifier(torch.nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model
        self.classifier = torch.nn.Linear(base_model.config.hidden_size, 4)
        
    def forward(self, **inputs):
        outputs = self.base_model(**inputs)
        return self.classifier(outputs.last_hidden_state[:, 0])

# Initialize model
base_model = AutoModelForSequenceClassification.from_pretrained(model_name)
classifier = ConversationClassifier(base_model)

# Define your categories
categories = [
    "accident_care",
    "road_assistance",
    "needs_clarification",
    "prank_call"
]

# Training data preparation (few-shot examples)
training_examples = [
    ("I crashed my car near Syntagma Square", "accident_care"),
    ("My engine won't start", "road_assistance"),
    ("Something happened to my vehicle", "needs_clarification"),
    ("LOL I just made up this story", "prank_call"),
    # Add more of your real examples here
]

# Convert to model inputs
def prepare_examples(examples):
    texts = [ex[0] for ex in examples]
    labels = [categories.index(ex[1]) for ex in examples]
    encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
    return encodings, torch.tensor(labels)

train_encodings, train_labels = prepare_examples(training_examples)

# Few-shot training
optimizer = torch.optim.AdamW(classifier.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

dataset = torch.utils.data.TensorDataset(
    train_encodings['input_ids'],
    train_encodings['attention_mask'],
    train_labels
)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

# Train for a few epochs
classifier.train()
for epoch in range(3):
    for batch in dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = classifier(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

# Classification function using the lightweight model
def classify_conversation(conversation: List[Dict]) -> str:
    # Extract user messages
    user_text = " ".join([msg["content"] for msg in conversation if msg["role"] == "user"])
    
    # Tokenize and predict
    inputs = tokenizer(user_text, return_tensors="pt", truncation=True, max_length=512)
    classifier.eval()
    with torch.no_grad():
        outputs = classifier(**inputs)
    
    # Get predicted category
    predicted_idx = torch.argmax(outputs).item()
    return categories[predicted_idx]

# Now use this in your data generation
def generate_balanced_dataset_with_llm(target_per_category=25):
    category_counts = {cat: 0 for cat in categories}
    synthetic_data = []
    
    while min(category_counts.values()) < target_per_category:
        new_conv = generate_synthetic_conversation()
        category = classify_conversation(new_conv)
        
        if category_counts[category] < target_per_category:
            synthetic_data.append(new_conv)
            category_counts[category] += 1
            print(f"Generated {len(synthetic_data)}: {category}")
    
    return synthetic_data

# Generate the dataset
balanced_data = generate_balanced_dataset_with_llm()

In [59]:
import random
import json
import pickle
from faker import Faker
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from tqdm import tqdm

# Initialize tools
fake = Faker('el_GR')  # Greek locale for realistic names/locations
device = "cuda" if torch.cuda.is_available() else "cpu"

# 1. Generate 100+ Synthetic Conversations
def generate_base_conversations():
    base_conversations = []
    
    # Accident Care Examples (30)
    for _ in range(30):
        name = fake.name()
        location = random.choice(["Syntagma Square", "Thessaloniki Center", "Patras Harbor"])
        accident_type = random.choice(["rear-end collision", "side impact", "head-on collision"])
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": f"I just had a {accident_type} accident near {location}! My name is {name}."},
                {"role": "assistant", "content": f"I'm sorry to hear about the accident Mr./Ms. {name.split()[-1]}. Are there any injuries?"},
                {"role": "user", "content": random.choice(["No injuries but the car is damaged", "Yes, we need an ambulance"])},
                {"role": "assistant", "content": f"We'll dispatch accident care to {location}. Is your car drivable?"}
            ],
            "category": "accident_care"
        })
    
    # Road Assistance Examples (30)
    for _ in range(30):
        name = fake.name()
        car_type = random.choice(["Toyota", "Honda", "BMW", "Mercedes"])
        issue = random.choice(["won't start", "has a flat tire", "is overheating"])
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": f"My {car_type} {issue} on {fake.street_name()} Street. I'm {name}."},
                {"role": "assistant", "content": f"I understand you need assistance Mr./Ms. {name.split()[-1]}. What's your exact location?"},
                {"role": "user", "content": f"Near the {random.choice(['bank', 'supermarket', 'park'])} on {fake.street_name()} Street"},
                {"role": "assistant", "content": "We'll send roadside assistance within 45 minutes. Is the car in a safe location?"}
            ],
            "category": "road_assistance"
        })
    
    # Needs Clarification Examples (20)
    for _ in range(20):
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": random.choice([
                    "My car has a problem",
                    "Something happened to my vehicle",
                    "I need help with my car"
                ])},
                {"role": "assistant", "content": "I'll help you. Could you please describe what happened with your vehicle?"},
                {"role": "user", "content": random.choice([
                    "It's not working right",
                    "I'm not sure, it just stopped",
                    "There was a noise"
                ])},
                {"role": "assistant", "content": "To better assist you, could you tell me your location and vehicle model?"}
            ],
            "category": "needs_clarification"
        })
    
    # Prank Call Examples (20)
    for _ in range(20):
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": random.choice([
                    "LOL my car just flew away!",
                    "I crashed into a unicorn",
                    "Just kidding, testing your service",
                    "My toaster is stuck in the car"
                ])},
                {"role": "assistant", "content": "This appears to be a non-serious inquiry. Please call back for genuine assistance."}
            ],
            "category": "prank_call"
        })
    
    return base_conversations

# Generate 100+ conversations
all_conversations = generate_base_conversations()
print(f"Generated {len(all_conversations)} conversations")

# 2. Train and Save Classifier Model
def train_and_save_classifier(conversations, save_path="/kaggle/working/insurance_classifier.pkl"):
    # Prepare training data
    texts = []
    labels = []
    for conv in conversations:
        user_text = " ".join([msg["content"] for msg in conv["messages"] if msg["role"] == "user"])
        texts.append(user_text)
        labels.append(conv["category"])
    
    # Initialize model
    model_name = "distilbert-base-multilingual-cased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=4,
        id2label={0: "accident_care", 1: "road_assistance", 2: "needs_clarification", 3: "prank_call"},
        label2id={"accident_care": 0, "road_assistance": 1, "needs_clarification": 2, "prank_call": 3}
    ).to(device)
    
    # Tokenize data
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
    input_ids = encodings["input_ids"].to(device)
    attention_mask = encodings["attention_mask"].to(device)
    labels = torch.tensor([model.config.label2id[label] for label in labels]).to(device)
    
    # Train for a few epochs
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    loss_fn = torch.nn.CrossEntropyLoss()
    
    model.train()
    for epoch in range(3):
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1} Loss: {loss.item()}")
    
    # Save model
    torch.save({
        "model_state_dict": model.state_dict(),
        "tokenizer": tokenizer,
        "config": model.config
    }, save_path)
    
    print(f"Model saved to {save_path}")
    return model, tokenizer

# Train and save model
train_and_save_classifier(all_conversations)

# 3. Save conversations to Kaggle
with open("/kaggle/working/insurance_conversations.json", "w") as f:
    json.dump(all_conversations, f, indent=2)

print("All files saved in Kaggle working directory:")
!ls /kaggle/working/

ModuleNotFoundError: No module named 'faker'

In [61]:
import random
import json
import pickle
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import os

# 1. Generate 100+ Synthetic Conversations without Faker
def generate_greek_names(count=50):
    first_names = ["Dimitris", "Maria", "Yannis", "Eleni", "Nikos", 
                  "Sophia", "Andreas", "Katerina", "George", "Anna"]
    last_names = ["Papadopoulos", "Ioannou", "Georgiou", "Karagiannis", 
                 "Nikolaou", "Antoniou", "Vasileiou", "Pappas", "Oikonomou"]
    return [f"{random.choice(first_names)} {random.choice(last_names)}" for _ in range(count)]

def generate_locations():
    return ["Syntagma Square, Athens", "Thessaloniki City Center", "Patras Harbor", 
           "Heraklion Central", "Larissa Main Street", "Volos Waterfront", 
           "Corfu Old Town", "Rhodes Medieval City", "Chania Venetian Port"]

def generate_base_conversations():
    greek_names = generate_greek_names()
    locations = generate_locations()
    base_conversations = []
    
    # Categories and their counts
    categories = {
        "accident_care": 30,
        "road_assistance": 30,
        "needs_clarification": 20,
        "prank_call": 20
    }
    
    # Accident Care Examples
    for _ in range(categories["accident_care"]):
        name = random.choice(greek_names)
        location = random.choice(locations)
        accident_type = random.choice(["rear-end collision", "side impact", "head-on collision"])
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": f"I just had a {accident_type} accident near {location}! My name is {name}."},
                {"role": "assistant", "content": f"I'm sorry to hear about the accident Mr./Ms. {name.split()[-1]}. Are there any injuries?"}
            ],
            "category": "accident_care"
        })
    
    # Road Assistance Examples
    for _ in range(categories["road_assistance"]):
        name = random.choice(greek_names)
        car_type = random.choice(["Toyota", "Honda", "BMW", "Mercedes"])
        issue = random.choice(["won't start", "has a flat tire", "is overheating"])
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": f"My {car_type} {issue} on {random.choice(locations)}. I'm {name}."},
                {"role": "assistant", "content": f"I understand you need assistance Mr./Ms. {name.split()[-1]}. What's your exact location?"}
            ],
            "category": "road_assistance"
        })
    
    # Needs Clarification Examples
    for _ in range(categories["needs_clarification"]):
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": random.choice([
                    "My car has a problem",
                    "Something happened to my vehicle",
                    "I need help with my car"
                ])},
                {"role": "assistant", "content": "I'll help you. Could you please describe what happened with your vehicle?"}
            ],
            "category": "needs_clarification"
        })
    
    # Prank Call Examples
    for _ in range(categories["prank_call"]):
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": random.choice([
                    "LOL my car just flew away!",
                    "I crashed into a unicorn",
                    "Just kidding, testing your service"
                ])},
                {"role": "assistant", "content": "This appears to be a non-serious inquiry. Please call back for genuine assistance."}
            ],
            "category": "prank_call"
        })
    
    return base_conversations

# Generate 100+ conversations
all_conversations = generate_base_conversations()
print(f"Generated {len(all_conversations)} conversations")

# 2. Train and Save Classifier Model
def train_and_save_classifier(conversations):
    # First install required packages
    !pip install transformers torch
    
    # Prepare training data
    texts = []
    labels = []
    for conv in conversations:
        user_text = " ".join([msg["content"] for msg in conv["messages"] if msg["role"] == "user"])
        texts.append(user_text)
        labels.append(conv["category"])
    
    # Initialize model
    model_name = "distilbert-base-uncased"  # Smaller than multilingual version
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=4,
        id2label={0: "accident_care", 1: "road_assistance", 2: "needs_clarification", 3: "prank_call"},
        label2id={"accident_care": 0, "road_assistance": 1, "needs_clarification": 2, "prank_call": 3}
    )
    
    # Tokenize data
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
    input_ids = encodings["input_ids"]
    attention_mask = encodings["attention_mask"]
    labels = torch.tensor([model.config.label2id[label] for label in labels])
    
    # Train for a few epochs
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    loss_fn = torch.nn.CrossEntropyLoss()
    
    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1} Loss: {loss.item()}")
    
    # Save model
    save_path = "/kaggle/working/insurance_classifier"
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    
    print(f"Model saved to {save_path}")
    return model, tokenizer

# Train and save model
model, tokenizer = train_and_save_classifier(all_conversations)

# 3. Save conversations to Kaggle
with open("/kaggle/working/insurance_conversations.json", "w") as f:
    json.dump(all_conversations, f, indent=2)

print("\nAll files saved in Kaggle working directory:")
!ls /kaggle/working/

Generated 100 conversations


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 Loss: 1.3806347846984863
Epoch 2 Loss: 1.2834265232086182
Epoch 3 Loss: 1.1761353015899658
Epoch 4 Loss: 1.1018908023834229
Epoch 5 Loss: 1.0093439817428589
Epoch 6 Loss: 0.9120502471923828
Epoch 7 Loss: 0.8360320925712585
Epoch 8 Loss: 0.7576269507408142
Epoch 9 Loss: 0.6863073110580444
Epoch 10 Loss: 0.6137562990188599
Epoch 11 Loss: 0.5571959614753723
Epoch 12 Loss: 0.4840092062950134
Epoch 13 Loss: 0.4266722500324249
Epoch 14 Loss: 0.3654012382030487
Epoch 15 Loss: 0.3237764835357666
Epoch 16 Loss: 0.2659647464752197
Epoch 17 Loss: 0.23127581179141998
Epoch 18 Loss: 0.19356413185596466
Epoch 19 Loss: 0.16371271014213562
Epoch 20 Loss: 0.14034779369831085
Epoch 21 Loss: 0.11583144962787628
Epoch 22 Loss: 0.10507556051015854
Epoch 23 Loss: 0.0874115452170372
Epoch 24 Loss: 0.07756716758012772
Epoch 25 Loss: 0.0688028559088707
Epoch 26 Loss: 0.059126969426870346
Epoch 27 Loss: 0.05377170443534851
Epoch 28 Loss: 0.04825491085648537
Epoch 29 Loss: 0.042238205671310425
Epoch 30 L

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [62]:
# Test examples
test_cases = [
    "I crashed my car near Syntagma Square",
    "My Toyota won't start",
    "Something's wrong with my vehicle",
    "LOL just kidding about my car"
]

print("\nModel Testing Results:")
for test in test_cases:
    prediction = classify_message(test, model, tokenizer, label_map)
    print(f"'{test}' => {prediction}")


Model Testing Results:


NameError: name 'classify_message' is not defined

In [66]:
import random
import json
import pickle
import zipfile
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# 1. Generate Synthetic Conversations
def generate_greek_names(count=50):
    first_names = ["Dimitris", "Maria", "Yannis", "Eleni", "Nikos", "Sophia", "Andreas", "Katerina", "George", "Anna"]
    last_names = ["Papadopoulos", "Ioannou", "Georgiou", "Karagiannis", "Nikolaou", "Antoniou", "Vasileiou", "Pappas", "Oikonomou"]
    return [f"{random.choice(first_names)} {random.choice(last_names)}" for _ in range(count)]

def generate_locations():
    return ["Syntagma Square, Athens", "Thessaloniki City Center", "Patras Harbor", 
            "Heraklion Central", "Larissa Main Street", "Volos Waterfront", 
            "Corfu Old Town", "Rhodes Medieval City", "Chania Venetian Port"]

def generate_base_conversations():
    greek_names = generate_greek_names()
    locations = generate_locations()
    base_conversations = []

    categories = {
        "accident_care": 30,
        "road_assistance": 30,
        "needs_clarification": 20,
        "prank_call": 20
    }

    for _ in range(categories["accident_care"]):
        name = random.choice(greek_names)
        location = random.choice(locations)
        accident_type = random.choice(["rear-end collision", "side impact", "head-on collision"])
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": f"I just had a {accident_type} accident near {location}! My name is {name}."},
                {"role": "assistant", "content": f"I'm sorry to hear about the accident Mr./Ms. {name.split()[-1]}. Are there any injuries?"}
            ],
            "category": "accident_care"
        })

    for _ in range(categories["road_assistance"]):
        name = random.choice(greek_names)
        car_type = random.choice(["Toyota", "Honda", "BMW", "Mercedes"])
        issue = random.choice(["won't start", "has a flat tire", "is overheating"])
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": f"My {car_type} {issue} on {random.choice(locations)}. I'm {name}."},
                {"role": "assistant", "content": f"I understand you need assistance Mr./Ms. {name.split()[-1]}. What's your exact location?"}
            ],
            "category": "road_assistance"
        })

    for _ in range(categories["needs_clarification"]):
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": random.choice([
                    "My car has a problem",
                    "Something happened to my vehicle",
                    "I need help with my car"
                ])},
                {"role": "assistant", "content": "I'll help you. Could you please describe what happened with your vehicle?"}
            ],
            "category": "needs_clarification"
        })

    for _ in range(categories["prank_call"]):
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": random.choice([
                    "LOL my car just flew away!",
                    "I crashed into a unicorn",
                    "Just kidding, testing your service"
                ])},
                {"role": "assistant", "content": "This appears to be a non-serious inquiry. Please call back for genuine assistance."}
            ],
            "category": "prank_call"
        })

    return base_conversations

all_conversations = generate_base_conversations()

# 2. Train and Save the Model
def train_and_save_classifier(conversations, save_path="/kaggle/working/insurance_classifier"):
    texts = [" ".join([msg["content"] for msg in conv["messages"] if msg["role"] == "user"]) for conv in conversations]
    labels = [conv["category"] for conv in conversations]

    label2id = {"accident_care": 0, "road_assistance": 1, "needs_clarification": 2, "prank_call": 3}
    id2label = {v: k for k, v in label2id.items()}

    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=4,
        id2label=id2label,
        label2id=label2id
    )

    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
    input_ids = encodings["input_ids"]
    attention_mask = encodings["attention_mask"]
    targets = torch.tensor([label2id[label] for label in labels])

    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    loss_fn = torch.nn.CrossEntropyLoss()

    model.train()
    for epoch in range(20):  # use 5 for speed, increase if needed
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    # Save model and tokenizer
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    # Zip model
    zip_path = save_path + ".zip"
    with zipfile.ZipFile(zip_path, "w") as zipf:
        for folder, _, files in os.walk(save_path):
            for file in files:
                file_path = os.path.join(folder, file)
                zipf.write(file_path, arcname=os.path.relpath(file_path, save_path))

    print(f"Model and tokenizer zipped to {zip_path}")
    return model, tokenizer

model, tokenizer = train_and_save_classifier(all_conversations)

# 3. Save Conversations
with open("/kaggle/working/insurance_conversations.json", "w") as f:
    json.dump(all_conversations, f, indent=2)

# 4. Test the Model
def predict_category(text, model, tokenizer):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted = torch.argmax(logits, dim=1).item()
    return model.config.id2label[predicted]

test_inputs = [
    "My car broke down on the highway",
    "I hit a pole in Thessaloniki",
    "Help me, something happened to my engine",
    "I just flew over Mount Olympus in my Ferrari lol"
]

print("\nTest Predictions:")
for input_text in test_inputs:
    category = predict_category(input_text, model, tokenizer)
    print(f"> \"{input_text}\" → {category}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 1.3926774263381958
Epoch 2, Loss: 1.3170080184936523
Epoch 3, Loss: 1.2257301807403564
Epoch 4, Loss: 1.1367565393447876
Epoch 5, Loss: 1.030173897743225
Epoch 6, Loss: 0.9215638637542725
Epoch 7, Loss: 0.8287941813468933
Epoch 8, Loss: 0.7390813231468201
Epoch 9, Loss: 0.6663830280303955
Epoch 10, Loss: 0.5982199311256409
Epoch 11, Loss: 0.5208102464675903
Epoch 12, Loss: 0.4686911404132843
Epoch 13, Loss: 0.40472543239593506
Epoch 14, Loss: 0.3505100905895233
Epoch 15, Loss: 0.30248579382896423
Epoch 16, Loss: 0.25744807720184326
Epoch 17, Loss: 0.22002579271793365
Epoch 18, Loss: 0.18347731232643127
Epoch 19, Loss: 0.1562412977218628
Epoch 20, Loss: 0.13636337220668793
Model and tokenizer zipped to /kaggle/working/insurance_classifier.zip

Test Predictions:
> "My car broke down on the highway" → needs_clarification
> "I hit a pole in Thessaloniki" → prank_call
> "Help me, something happened to my engine" → prank_call
> "I just flew over Mount Olympus in my Ferrari lol

In [65]:
import random
import json
import pickle
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import os
from zipfile import ZipFile

# 1. Generate Synthetic Conversations (same as before)
def generate_greek_names(count=50):
    first_names = ["Dimitris", "Maria", "Yannis", "Eleni", "Nikos", 
                  "Sophia", "Andreas", "Katerina", "George", "Anna"]
    last_names = ["Papadopoulos", "Ioannou", "Georgiou", "Karagiannis", 
                 "Nikolaou", "Antoniou", "Vasileiou", "Pappas", "Oikonomou"]
    return [f"{random.choice(first_names)} {random.choice(last_names)}" for _ in range(count)]

def generate_locations():
    return ["Syntagma Square, Athens", "Thessaloniki City Center", "Patras Harbor", 
           "Heraklion Central", "Larissa Main Street", "Volos Waterfront", 
           "Corfu Old Town", "Rhodes Medieval City", "Chania Venetian Port"]

def generate_base_conversations():
    greek_names = generate_greek_names()
    locations = generate_locations()
    base_conversations = []
    
    categories = {
        "accident_care": 30,
        "road_assistance": 30,
        "needs_clarification": 20,
        "prank_call": 20
    }
    
    # [Previous conversation generation code remains exactly the same]
    # ... (keep all your existing conversation generation code)

# Generate conversations
all_conversations = generate_base_conversations()

# 2. Enhanced Model Training with Evaluation
def train_and_save_classifier(conversations):
    # Prepare data
    texts = [" ".join(m["content"] for m in conv["messages"] if m["role"] == "user"] for conv in conversations]
    labels = [conv["category"] for conv in conversations]
    
    # Initialize model
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=4,
        id2label={0: "accident_care", 1: "road_assistance", 2: "needs_clarification", 3: "prank_call"},
        label2id={"accident_care": 0, "road_assistance": 1, "needs_clarification": 2, "prank_call": 3}
    )
    
    # Tokenize
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
    input_ids = encodings["input_ids"]
    attention_mask = encodings["attention_mask"]
    labels = torch.tensor([model.config.label2id[label] for label in labels])
    
    # Train with progress tracking
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    model.train()
    for epoch in range(3):  # Reduced epochs for Kaggle
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")
    
    # Save in multiple formats
    save_dir = "/kaggle/working/insurance_model"
    os.makedirs(save_dir, exist_ok=True)
    
    # Format 1: Native Hugging Face format
    model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)
    
    # Format 2: Pickle file
    with open(f"{save_dir}/model.pkl", "wb") as f:
        pickle.dump({
            "model_state": model.state_dict(),
            "tokenizer": tokenizer,
            "config": model.config
        }, f)
    
    # Format 3: Compressed ZIP
    with ZipFile(f"{save_dir}/model.zip", 'w') as zipf:
        for root, _, files in os.walk(save_dir):
            for file in files:
                if file.endswith('.pkl') or file.endswith('.json'):
                    zipf.write(os.path.join(root, file), file)
    
    return model, tokenizer

# Train and save
model, tokenizer = train_and_save_classifier(all_conversations)

# 3. Save conversations
with open("/kaggle/working/insurance_conversations.json", "w") as f:
    json.dump(all_conversations, f, indent=2)

# 4. Model Testing
test_questions = [
    "I crashed my car near the airport",
    "My Honda won't start in Thessaloniki",
    "Something's wrong with my vehicle",
    "LOL my car turned into a banana",
    "There was a collision on the highway",
    "Just testing your system",
    "My BMW is overheating",
    "I need help with my Fiat"
]

def test_model(model, tokenizer, questions):
    print("\n" + "="*50)
    print("MODEL TESTING RESULTS")
    print("="*50)
    model.eval()
    for question in questions:
        inputs = tokenizer(question, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        pred = model.config.id2label[outputs.logits.argmax().item()]
        print(f"Question: {question[:60]}...")
        print(f"Prediction: {pred}\n")

test_model(model, tokenizer, test_questions)

# 5. Prepare Downloadable Files
print("\n" + "="*50)
print("DOWNLOADABLE FILES")
print("="*50)
print("1. insurance_model/ - Full model directory")
print("2. insurance_model/model.pkl - Pickle format")
print("3. insurance_model/model.zip - Compressed bundle")
print("4. insurance_conversations.json - All training data")

# Create download instructions
with open("/kaggle/working/README.txt", "w") as f:
    f.write("""HOW TO DOWNLOAD:
1. Click the folder icon on the left
2. Navigate to /kaggle/working/
3. Check the files you want
4. Click the 'Download' button

Files available:
- insurance_model/ (full model)
- insurance_model/model.pkl (pickle version)
- insurance_model/model.zip (compressed)
- insurance_conversations.json (training data)
""")

print("\nTo download: Check /kaggle/working/ in the file browser and click Download")

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (1562594428.py, line 43)

In [72]:
import random
import json
import pickle
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import os
from zipfile import ZipFile

# 1. Generate Synthetic Conversations
def generate_greek_names(count=50):
    first_names = ["Dimitris", "Maria", "Yannis", "Eleni", "Nikos", 
                  "Sophia", "Andreas", "Katerina", "George", "Anna"]
    last_names = ["Papadopoulos", "Ioannou", "Georgiou", "Karagiannis", 
                 "Nikolaou", "Antoniou", "Vasileiou", "Pappas", "Oikonomou"]
    return [f"{random.choice(first_names)} {random.choice(last_names)}" for _ in range(count)]

def generate_locations():
    return ["Syntagma Square, Athens", "Thessaloniki City Center", "Patras Harbor", 
           "Heraklion Central", "Larissa Main Street", "Volos Waterfront", 
           "Corfu Old Town", "Rhodes Medieval City", "Chania Venetian Port"]

def generate_base_conversations():
    greek_names = generate_greek_names()
    locations = generate_locations()
    base_conversations = []
    
    categories = {
        "accident_care": 30,
        "road_assistance": 30,
        "needs_clarification": 20,
        "prank_call": 20
    }
    
    # Accident Care Examples
    for _ in range(categories["accident_care"]):
        name = random.choice(greek_names)
        location = random.choice(locations)
        accident_type = random.choice(["rear-end collision", "side impact", "head-on collision"])
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": f"I just had a {accident_type} accident near {location}! My name is {name}."},
                {"role": "assistant", "content": f"I'm sorry to hear about the accident Mr./Ms. {name.split()[-1]}. Are there any injuries?"}
            ],
            "category": "accident_care"
        })
    
    # Road Assistance Examples
    for _ in range(categories["road_assistance"]):
        name = random.choice(greek_names)
        car_type = random.choice(["Toyota", "Honda", "BMW", "Mercedes"])
        issue = random.choice(["won't start", "has a flat tire", "is overheating"])
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": f"My {car_type} {issue} on {random.choice(locations)}. I'm {name}."},
                {"role": "assistant", "content": f"I understand you need assistance Mr./Ms. {name.split()[-1]}. What's your exact location?"}
            ],
            "category": "road_assistance"
        })
    
    # Needs Clarification Examples
    for _ in range(categories["needs_clarification"]):
        base_conversations.append({
            "messages": [
                {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
                {"role": "user", "content": random.choice([
                    "My car has a problem",
                    "Something happened to my vehicle",
                    "I need help with my car"
                ])},
                {"role": "assistant", "content": "I'll help you. Could you please describe what happened with your vehicle?"}
            ],
            "category": "needs_clarification"
        })
    
    # Prank Call Examples
    for _ in range(categories["prank_call"]):
        base_conversations.append({
            "messages": [
                {"role": "system", "content": """You are an AI assistant for the insurance company named Greek Insurance Company.
                You will be contacted by different types of people when they have a problem with their car.
                Your first role from the message you receive from the client it to determine in what category they belong into.
                The four character categories are:
                1) Clients that need accident care aka have either been in a big accident, head-on collision or any type of accident that involves multiple vehicles.
                2) Clients that need road assistance aka the car has shown any kind of malfunction and does not move normall so the client cannot go to their intended destination
                3) Clients that have not provided enough information to determine whether they are one of the top two categories so we ask a question for clarification
                4) Clients that by their way of talking or mocking remarks or lack of any information after a couple of questions we determine are not serious callers and are doing a phone prank.
                 """},
                {"role": "user", "content": random.choice([
                    "LOL my car just flew away!",
                    "I crashed into a unicorn",
                    "Just kidding, testing your service"
                ])},
                {"role": "assistant", "content": "This appears to be a non-serious inquiry. Please call back for genuine assistance."}
            ],
            "category": "prank_call"
        })
    
    return base_conversations

# Generate conversations
all_conversations = generate_base_conversations()
print(f"Generated {len(all_conversations)} conversations")

# 2. Train and Save Classifier Model
def train_and_save_classifier(conversations):
    # Prepare training data - FIXED THE SYNTAX ERROR HERE
    texts = []
    labels = []
    for conv in conversations:
        user_messages = [m["content"] for m in conv["messages"] if m["role"] == "user"]
        texts.append(" ".join(user_messages))
        labels.append(conv["category"])
    
    # Initialize model
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=4,
        id2label={0: "accident_care", 1: "road_assistance", 2: "needs_clarification", 3: "prank_call"},
        label2id={"accident_care": 0, "road_assistance": 1, "needs_clarification": 2, "prank_call": 3}
    )
    
    # Tokenize data
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
    input_ids = encodings["input_ids"]
    attention_mask = encodings["attention_mask"]
    labels = torch.tensor([model.config.label2id[label] for label in labels])
    
    # Train with progress tracking
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    model.train()
    for epoch in range(50):
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")
    
    # Save in multiple formats
    save_dir = "/kaggle/working/insurance_model"
    os.makedirs(save_dir, exist_ok=True)
    
    # Format 1: Native Hugging Face format
    model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)
    
    # Format 2: Pickle file
    with open(f"{save_dir}/model.pkl", "wb") as f:
        pickle.dump({
            "model_state": model.state_dict(),
            "tokenizer": tokenizer,
            "config": model.config
        }, f)
    
    # Format 3: Compressed ZIP
    with ZipFile(f"{save_dir}/model.zip", 'w') as zipf:
        for root, _, files in os.walk(save_dir):
            for file in files:
                if file.endswith('.pkl') or file.endswith('.json'):
                    zipf.write(os.path.join(root, file), file)
    
    return model, tokenizer

# Train and save
model, tokenizer = train_and_save_classifier(all_conversations)

# 3. Save conversations
with open("/kaggle/working/insurance_conversations.json", "w") as f:
    json.dump(all_conversations, f, indent=2)

# 4. Model Testing
test_questions = [
    "I crashed my car near the airport",
    "My Honda won't start in Thessaloniki",
    "Something's wrong with my vehicle",
    "LOL my car turned into a banana",
    "There was a collision on the highway",
    "Just testing your system",
    "My BMW is overheating",
    "I need help with my Fiat"
]

def test_model(model, tokenizer, questions):
    print("\n" + "="*50)
    print("MODEL TESTING RESULTS")
    print("="*50)
    model.eval()
    for question in questions:
        inputs = tokenizer(question, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        pred = model.config.id2label[outputs.logits.argmax().item()]
        print(f"Question: {question[:60]}...")
        print(f"Prediction: {pred}\n")

test_model(model, tokenizer, test_questions)

# 5. Prepare Downloadable Files
print("\n" + "="*50)
print("DOWNLOADABLE FILES")
print("="*50)
print("1. insurance_model/ - Full model directory")
print("2. insurance_model/model.pkl - Pickle format")
print("3. insurance_model/model.zip - Compressed bundle")
print("4. insurance_conversations.json - All training data")

# Create download instructions
with open("/kaggle/working/README.txt", "w") as f:
    f.write("""HOW TO DOWNLOAD:
1. Click the folder icon on the left
2. Navigate to /kaggle/working/
3. Check the files you want
4. Click the 'Download' button

Files available:
- insurance_model/ (full model)
- insurance_model/model.pkl (pickle version)
- insurance_model/model.zip (compressed)
- insurance_conversations.json (training data)
""")

print("\nTo download: Check /kaggle/working/ in the file browser and click Download")

Generated 100 conversations


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 Loss: 1.3794
Epoch 2 Loss: 1.2819
Epoch 3 Loss: 1.1707
Epoch 4 Loss: 1.0769
Epoch 5 Loss: 0.9768
Epoch 6 Loss: 0.8736
Epoch 7 Loss: 0.7859
Epoch 8 Loss: 0.6806
Epoch 9 Loss: 0.6034
Epoch 10 Loss: 0.5310
Epoch 11 Loss: 0.4556
Epoch 12 Loss: 0.3908
Epoch 13 Loss: 0.3280
Epoch 14 Loss: 0.2724
Epoch 15 Loss: 0.2301
Epoch 16 Loss: 0.2100
Epoch 17 Loss: 0.1708
Epoch 18 Loss: 0.1502
Epoch 19 Loss: 0.1293
Epoch 20 Loss: 0.1113
Epoch 21 Loss: 0.0983
Epoch 22 Loss: 0.0850
Epoch 23 Loss: 0.0774
Epoch 24 Loss: 0.0674
Epoch 25 Loss: 0.0606
Epoch 26 Loss: 0.0551
Epoch 27 Loss: 0.0496
Epoch 28 Loss: 0.0457
Epoch 29 Loss: 0.0402
Epoch 30 Loss: 0.0391
Epoch 31 Loss: 0.0345
Epoch 32 Loss: 0.0319
Epoch 33 Loss: 0.0303
Epoch 34 Loss: 0.0268
Epoch 35 Loss: 0.0257
Epoch 36 Loss: 0.0246
Epoch 37 Loss: 0.0226
Epoch 38 Loss: 0.0216
Epoch 39 Loss: 0.0198
Epoch 40 Loss: 0.0196
Epoch 41 Loss: 0.0184
Epoch 42 Loss: 0.0184
Epoch 43 Loss: 0.0163
Epoch 44 Loss: 0.0157
Epoch 45 Loss: 0.0142
Epoch 46 Loss: 0.01

# Train the model

In [73]:
training_examples = [
    {
        "messages": [
            {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
            {"role": "assistant", "content": "Hello! How can I assist you?"},
            {"role": "user", "content": "Hello. I just had a car accident."},
            {"role": "assistant", "content": "I'm sorry to hear that! How long ago did it happen?"},
            {"role": "user", "content": "About fifteen minutes ago."},
            {"role": "assistant", "content": "I'd like to know if there are any injuries."},
            {"role": "user", "content": "No, fortunately, we are all fine. There’s only material damage."},
            {"role": "assistant", "content": "I'm glad to hear everyone is okay! I’ll need to collect some of your details. Let's start with your vehicle’s license plate number."},
            {"role": "user", "content": "The plate number is MNR-9012."},
            {"role": "assistant", "content": "Additionally, I’d like to know under whose name the insurance policy is registered."},
            {"role": "user", "content": "Under the name Pinelopi Papachristou."},
            {"role": "assistant", "content": "Very well! Am I speaking with her directly?"},
            {"role": "user", "content": "Yes, it’s me in person."},
            {"role": "assistant", "content": "Would you like to describe to us exactly how the accident happened?"},
            {"role": "user", "content": "I was stopped on Vouliagmenis Avenue, at the traffic light at the intersection with Zeppou Street, and a vehicle hit me from behind."},
            {"role": "assistant", "content": "Very helpful. I’d also like you to tell me exactly where the damage is located on your vehicle."},
            {"role": "user", "content": "On the rear light and the rear bumper."},
            {"role": "assistant", "content": "Understood. Do you need me to call roadside assistance?"},
            {"role": "user", "content": "Yes, I would like that."},
            {"role": "assistant", "content": "Of course, I’m calling roadside assistance to your location right away. Do you also have the details of the driver of the other vehicle involved in the collision (name, surname, phone number, insurance company)?"},
            {"role": "user", "content": "Yes. His name is Panagiotis Antoniou, his phone number is 6986345752 and his insurance company is Anytime."},
            {"role": "assistant", "content": "Do you know if Anytime has already been notified and whether the driver accepts responsibility for the incident?"},
            {"role": "user", "content": "Yes, both the insurance company has been notified and Mr. Antoniou accepts responsibility for the accident."},
            {"role": "assistant", "content": "Thank you for the information. The accident report is now complete, and roadside assistance will arrive at the scene shortly. Below you’ll find a discount coupon to use at nearby supermarkets while you wait for assistance."},
            {"role": "user", "content": "Thank you very much for your help!"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
            {"role": "assistant", "content": "Hello! How can I assist you?"},
            {"role": "user", "content": "Hi, my car won’t start and I’m stuck in the middle of the road."},
            {"role": "assistant", "content": "I’m sorry to hear that. How long have you been stopped there?"},
            {"role": "user", "content": "About 10 minutes."},
            {"role": "assistant", "content": "I’ll need your vehicle’s license plate number and your full name, please."},
            {"role": "user", "content": "The plate number is IMO1806 and my name is Ioannis Papadopoulos."},
            {"role": "assistant", "content": "Understood. Could you also give me your exact location?"},
            {"role": "user", "content": "Yes, 19-21 Damareos Street in Pagrati, between Kononos and Artotinis."},
            {"role": "assistant", "content": "I’ll call roadside assistance right away to come to your location. Is there anyone in the vehicle who needs special assistance or is there any other emergency?"},
            {"role": "user", "content": "No, fortunately everyone is fine."},
            {"role": "assistant", "content": "Glad to hear that. I’ve contacted roadside assistance and they’ll be at your location within 10 minutes. Below you’ll find a coupon you can use at nearby shops while you wait. Thank you very much for contacting us."},
            {"role": "user", "content": "Thank you very much as well."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
            {"role": "assistant", "content": "Hello! How can I assist you?"},
            {"role": "user", "content": "Hello, I just had a minor collision in the supermarket parking lot."},
            {"role": "assistant", "content": "I’m sorry to hear that. Are there any injuries?"},
            {"role": "user", "content": "No, fortunately we’re all fine. Just minor material damage."},
            {"role": "assistant", "content": "Alright, that’s the most important thing. I’d like you to give me your vehicle’s license plate number and your full name."},
            {"role": "user", "content": "The plate number is YRZ3456 and my name is Eleni Konstantinou."},
            {"role": "assistant", "content": "Understood. Could you give me your exact location and, if possible, the plate number of the other vehicle?"},
            {"role": "user", "content": "I’m at the AB Vasilopoulos parking lot on 123 Thivon Street, in Peristeri. The other vehicle’s plate number is ZKM7890."},
            {"role": "assistant", "content": "Thank you. I’ll contact the nearest roadside assistance service to come and collect your vehicle. It should be at your location in about 15–20 minutes. Below you’ll find a coupon you can use at nearby shops while you wait. Thank you very much for contacting us."},
            {"role": "user", "content": "Thank you as well."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
            {"role": "assistant", "content": "Hello! How can I assist you?"},
            {"role": "user", "content": "Hello, my car was damaged by the recent hailstorm."},
            {"role": "assistant", "content": "I'm sorry to hear that. Are there any injuries?"},
            {"role": "user", "content": "No, fortunately I was inside the house. Only the car has dents on the hood and roof."},
            {"role": "assistant", "content": "Understood. I’d like you to give me your vehicle’s license plate number and your full name."},
            {"role": "user", "content": "The license plate number is POM4567 and my name is Sofia Dimitriou."},
            {"role": "assistant", "content": "Got it. Could you tell me the exact location where the incident occurred and give me a brief description of the damage?"},
            {"role": "user", "content": "It was parked at 30 Evangelistrias Street, in Kifisia. There are several dents from the hail, mainly on the hood, roof, and trunk."},
            {"role": "assistant", "content": "I understand. I will immediately contact roadside assistance to have your vehicle taken to the nearest repair shop if you wish."},
            {"role": "user", "content": "Yes, please."},
            {"role": "assistant", "content": "Perfect. Roadside assistance will arrive shortly at your location. Below, you’ll find a coupon you can use at nearby shops while you wait. Thank you very much for contacting us."},
            {"role": "user", "content": "Thank you."}
        ]
    },
        {
        "messages": [
            {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
            {"role": "assistant", "content": "Good evening! How can I help you today?"},
            {"role": "user", "content": "Good evening, I had a minor car accident with another vehicle and I need assistance."},
            {"role": "assistant", "content": "I'm sorry to hear that. Are you and your passengers okay?"},
            {"role": "user", "content": "Yes, fortunately there are no injuries."},
            {"role": "assistant", "content": "Could you please tell me exactly where the accident happened?"},
            {"role": "user", "content": "On Athinas Street, near the intersection with Solonos Street."},
            {"role": "assistant", "content": "Thank you. I’ll also need your vehicle's license plate number and your name."},
            {"role": "user", "content": "The license plate number is ABG1234 and my name is Maria Konstantinou."},
            {"role": "assistant", "content": "Understood. I’ve recorded your details. I will forward your request for damage assessment and we will call you shortly with the next steps. Do you need roadside assistance?"},
            {"role": "user", "content": "No, the vehicle can be driven to the nearest repair shop."},
            {"role": "assistant", "content": "Excellent. Please make sure to keep the other party's details for the insurance process. Thank you for contacting us."},
            {"role": "user", "content": "Thank you very much as well."}
        ]
    },
        {
        "messages": [
            {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
            {"role": "assistant", "content": "Good evening! How can I help you today?"},
            {"role": "user", "content": "Good evening, my car has broken down due to an engine failure and it won't start."},
            {"role": "assistant", "content": "How long have you been stopped there?"},
            {"role": "user", "content": "About 15 minutes."},
            {"role": "assistant", "content": "Could you please give me your license plate number and full name?"},
            {"role": "user", "content": "The plate number is ZXY5678 and my name is Dimitris Papadopoulos."},
            {"role": "assistant", "content": "Could you also tell me your exact location?"},
            {"role": "user", "content": "I'm on Mesogeion Avenue 45, near the metro station."},
            {"role": "assistant", "content": "I will immediately call roadside assistance to come to your location. Is there anything that needs urgent attention, such as injured persons or passengers in immediate need?"},
            {"role": "user", "content": "Yes, I was transporting a patient to the Children's Hospital."},
            {"role": "assistant", "content": "Understood. I can call an ambulance or a taxi to take you to the Children's Hospital if you wish."},
            {"role": "user", "content": "Thank you very much for your help!"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are an AI assistant for Greek Insurance Company..."},
            {"role": "assistant", "content": "Good evening! How can I help you today?"},
            {"role": "user", "content": "Good evening. My car has broken down and I'm stranded on the National Highway."},
            {"role": "assistant", "content": "Could you please give me your vehicle's license plate number?"},
            {"role": "user", "content": "EIH2472"},
            {"role": "assistant", "content": "I see this is a 2015 Renault Clio, correct?"},
            {"role": "user", "content": "Yes, that's correct."},
            {"role": "assistant", "content": "Whose name is the insurance policy registered under?"},
            {"role": "user", "content": "It's under the name Michail Kouris."},
            {"role": "assistant", "content": "Are you Mr. Kouris?"},
            {"role": "user", "content": "No, I'm his son-in-law. I was driving it today."},
            {"role": "assistant", "content": "If you like, please give me your full name so I can note it in our records."},
            {"role": "user", "content": "My name is Dimitris Papageorgiou."},
            {"role": "assistant", "content": "Thank you very much, Mr. Papageorgiou. Shall we keep the phone number we're speaking on for future contact with our partner?"},
            {"role": "user", "content": "Yes, of course."},
            {"role": "assistant", "content": "Can you briefly describe the issue with the car?"},
            {"role": "user", "content": "The gas pedal went loose and doesn't respond at all, so we assume the throttle cable snapped."},
            {"role": "assistant", "content": "Understood. So we’ll arrange for a tow to a garage. Where exactly are you located right now?"},
            {"role": "user", "content": "I’m in Kallithea, on Sachtouri Street."},
            {"role": "assistant", "content": "Do you have a nearby shop or building number that could help our partner locate you?"},
            {"role": "user", "content": "There’s a Kotsovolos store directly across from me."},
            {"role": "assistant", "content": "Alright, we’ll note Sachtouri Street, across from Kotsovolos in Kallithea. What color is the vehicle?"},
            {"role": "user", "content": "It’s dark blue."},
            {"role": "assistant", "content": "Is there a specific garage you’d like the vehicle to be towed to?"},
            {"role": "user", "content": "Yes, it’s in Agios Dimitrios, on Agiou Dimitriou Street."},
            {"role": "assistant", "content": "Do you know the name of the garage?"},
            {"role": "user", "content": "It’s called 'Auto Michalis'."},
            {"role": "assistant", "content": "Very good. I found Michalis Patsias’ garage at 246 Agiou Dimitriou Street. Is that the one?"},
            {"role": "user", "content": "Yes, that’s exactly it."},
            {"role": "assistant", "content": "Great. Does the vehicle have any particular specifications? Automatic, lowered, carrying a load?"},
            {"role": "user", "content": "It’s automatic, but not lowered and not carrying any load."},
            {"role": "assistant", "content": "Can it be put into neutral for loading?"},
            {"role": "user", "content": "Yes, no problem."},
            {"role": "assistant", "content": "Very well! Our partner will be at your location within the next hour. If you’d like, you can accompany the vehicle to the garage or let the mechanic receive it there."},
            {"role": "user", "content": "The mechanic is already informed and waiting."},
            {"role": "assistant", "content": "Wonderful. In the meantime, since I see you have the Wolt app on your phone, we’ll send you a €5 discount code for your next order — a small gift while you wait."},
            {"role": "user", "content": "Thank you very much! That’s very thoughtful of you."},
            {"role": "assistant", "content": "With pleasure. I wish you a pleasant rest of your day, and we’re here for anything else you might need."},
            {"role": "user", "content": "You too, thank you for the service."}
        ]
    }
]

In [101]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Replace with any LLaMA model you have access to
model_id = "meta-llama/Llama-3.1-8B-Instruct"  # or Llama-3-8b-instruct if available

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True  # Save memory
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0.2,
    repetition_penalty=1.1
)  

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [121]:
from collections import defaultdict

# Add this at the top of your script
conversation_histories = defaultdict(list)

def update_chat_template(history):
    system_prompt = """
                You are an AI assistant for the insurance company named Greek Insurance Company.
                Your name is Michelle and you are there to help.
                You will be contacted by different types of people when they have a problem with their car.
                Your first role from the message you receive from the client it to determine in what category they belong into.
                The four character categories are:
                1) Clients that need accident care aka have either been in a big accident, head-on collision or any type of accident that involves multiple vehicles.
                2) Clients that need road assistance aka the car has shown any kind of malfunction and does not move normall so the client cannot go to their intended destination
                3) Clients that have not provided enough information to determine whether they are one of the top two categories so we ask a question for clarification
                4) Clients that by their way of talking or mocking remarks or lack of any information after a couple of questions we determine are not serious callers and are doing a phone prank.
                In the last case your response should be dismissive of our services.
                In the third case indirectly try to ask them what exactly happened so we get that information eventually.
                In the first two cases you should ask for as many of pieces of information listed below we can get.
                The information we want to hear from the client include
                - Full Name
                - Category of Message (as mentioned above)
                - Location of Accident/Malfunction
                - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
                call accident care to note the accident who was at fault etc.)
                - The type of location they should end up (service center, home etc.) 
                - The exact location of the above location point
                If you dont get one of the above info start by understanding the client and offering your apologies for what happened
                and then try to get as many of the other pieces of information without being overbearing
                Always be respectful and nice to the client.
                Be very clear with your answers to the client.
                The ONLY thing you should return is the message towards the client NOTHING MORE
                Don't repeat to the client the info they gave you in previous message
                If you get conflicting information ask for clarification on what is the correct info, precision is important
                Also don't over ask the client if you have all information available and some more it is ok to stop asking
                Always care about their safety
                Try to keep the conversations short and precise and to the point most conversations should last up to 7 messages unless you don't have yet some of the info needed
                Try to diagnose the problem if they talk about why their car broke down or stopped, use your knowledge about what can cause certain accidents
                
                
                Client says: {query}
                
                Your reply:
    """

    messages = [{"role": "system", "content": system_prompt}]
    for i, (user_msg, assistant_msg) in enumerate(zip(history[::2], history[1::2])):
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})
    if len(history) % 2 == 1:
        messages.append({"role": "user", "content": history[-1]})
    
    return tokenizer.apply_chat_template(messages, tokenize=False)

def insurance_chat(query, user_id="default"):
    # Get conversation history
    history = conversation_histories[user_id]
    history.append(query)
    
    # Generate prompt with history
    prompt = update_chat_template(history)
    
    # Generate response
    response_raw = pipe(prompt)[0]['generated_text']
    
    # Extract just the new response
    response = response_raw.split(history[-1])[-1].strip()
    
    # Update history with assistant response
    history.append(response)
    
    # Generate structured extraction (modified to use full conversation)
    extract_prompt = f"""
        Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - The type of location they should end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {{
          "name": "...",
          "category": "...",
          "location_accident": "...",
          "description": "...",
          "type_of_final_location": "...",
          
        }}
        Note: add more rows in the json file as you judge you get more and more important info
        Client message: {query}
        The history is the following: {" ".join(history)}"""
    
    extracted_raw = pipe(extract_prompt)[0]['generated_text'].strip()
    
    try:
        extracted_json = json.loads(extracted_raw)
    except json.JSONDecodeError:
        print("⚠️ JSON parsing failed. Raw output:\n", extracted_raw)
        extracted_json = None

    print("🤖 Assistant:\n", response)
    print("\n📄 Extracted Info:\n", extracted_json if extracted_json else extracted_raw)
    
    return response, extracted_json

In [117]:
!pip install gradio --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 110.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00


In [115]:
insurance_chat("Hi, my name is Alice Chen. My car has broke down on Langston Street a few hours ago can you help?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - The type of location they should end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "category": "...",
          "location_accident": "...",
          "description": "...",
          "type_of_final_location": "...",
          
        }
        Note: add more rows in the json file as you judge you get more and more

("<|eot_id|>assistant\n\nHello Alice, I'm glad you reached out to us. Sorry to hear that your car broke down on Langston Street. Can you tell me a bit more about what's going on? Did your car just stall or did you experience something else like a loss of power steering or brakes?\n\nAlso, would you like Road Assistance to come and help you get back on the road, or would you prefer to have your vehicle towed to a Service Center for further inspection?",
 None)

In [119]:
import gradio as gr

# Assume your insurance_chat() and pipe are already defined above
# Also assume conversation_histories is already initialized

# Define your chatbot interface function
def gradio_chat(message, history):
    response, _ = insurance_chat(message)
    return response

# Define a function to reset the history
def reset_history():
    conversation_histories["default"] = []
    return gr.update(value=None), gr.update(value=None)

# Build the UI
with gr.Blocks(title="Insurance Bot Chat") as demo:
    gr.Markdown("## 💼 Insurance Chatbot\nTalk to a fine-tuned model about your insurance issues.")

    chatbot = gr.Chatbot(label="Chat", height=400)
    msg = gr.Textbox(label="Type your message here...", placeholder="E.g. I had an accident in San Francisco...")

    with gr.Row():
        submit_btn = gr.Button("Submit", variant="primary")
        clear_btn = gr.Button("🗑️ New Chat / Reset History")

    # Logic to process new messages
    def user_submit(user_message, chat_history):
        response = gradio_chat(user_message, chat_history)
        conversation_histories["default"].append(user_message)
        conversation_histories["default"].append(response)
        chat_history = chat_history + [[user_message, response]]
        return "", chat_history

    submit_btn.click(user_submit, [msg, chatbot], [msg, chatbot])
    msg.submit(user_submit, [msg, chatbot], [msg, chatbot])  # pressing Enter also submits

    # Logic to reset the whole convo
    clear_btn.click(fn=reset_history, outputs=[msg, chatbot])

# Run it!
demo.launch()

/tmp/ipykernel_35/2032308883.py:20: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat", height=400)


* Running on local URL:  http://127.0.0.1:7861
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://5a58e2fa03c49c286b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1965: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  state[block._id] = block.__class__(**kwargs)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - The type of location they should end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "category": "...",
          "location_accident": "...",
          "description": "...",
          "type_of_final_location": "...",
          
        }
        Note: add more rows in the json file as you judge you get more and more

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - The type of location they should end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "category": "...",
          "location_accident": "...",
          "description": "...",
          "type_of_final_location": "...",
          
        }
        Note: add more rows in the json file as you judge you get more and more

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - The type of location they should end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "category": "...",
          "location_accident": "...",
          "description": "...",
          "type_of_final_location": "...",
          
        }
        Note: add more rows in the json file as you judge you get more and more

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - The type of location they should end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "category": "...",
          "location_accident": "...",
          "description": "...",
          "type_of_final_location": "...",
          
        }
        Note: add more rows in the json file as you judge you get more and more

In [125]:
import gradio as gr

# 🌐 Avatar image links — swap these for your own
USER_AVATAR = "https://img.freepik.com/premium-vector/avatar-icon0002_750950-43.jpg"
BOT_AVATAR = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSMzp4cfvB5JfHaMgMzcJSlDSoSj3g0CDQqIA&s"

# 🌈 Consistent color palette
USER_COLOR = "#15314b"  # Dark navy blue
BOT_COLOR = "#1e3a5f"   # Slightly lighter dark blue

# 🧱 Format messages with circular avatars + styled bubbles
def format_message(message, is_user):
    avatar = USER_AVATAR if is_user else BOT_AVATAR
    bg_color = USER_COLOR if is_user else BOT_COLOR
    align = "flex-start" if is_user else "flex-end"
    direction = "row" if is_user else "row-reverse"

    return f"""
    <div style="display: flex; flex-direction: {direction}; align-items: flex-start; justify-content: {align}; margin: 10px 0;">
        <img src="{avatar}" width="40" height="40" style="border-radius: 50%; margin: 0 10px;">
        <div style="background: {bg_color}; color: white; padding: 10px 15px; border-radius: 15px; max-width: 65%; word-wrap: break-word;">
            {message}
        </div>
    </div>
    """

# 🧹 Reset backend history
def reset_history():
    conversation_histories["default"] = []
    return [], ""

# 🔁 Handle user message + bot response
def user_submit(user_message, chat_display):
    response = gradio_chat(user_message, chat_display)
    conversation_histories["default"].append(user_message)
    conversation_histories["default"].append(response)

    user_msg_html = format_message(user_message, is_user=True)
    bot_msg_html = format_message(response, is_user=False)

    updated_display = chat_display + user_msg_html + bot_msg_html
    return "", updated_display

# 🎨 Main UI
with gr.Blocks(title="Sleek Insurance Chatbot") as demo:
    gr.Markdown("## 🤖 Insurance Chatbot\nTalk to your AI-powered insurance assistant.")

    chat_display = gr.HTML()
    
    with gr.Row():
        user_input = gr.Textbox(placeholder="Ask me anything about your insurance...", show_label=False, scale=9)
        submit_btn = gr.Button("📤", elem_id="send-btn", scale=1)

    reset_btn = gr.Button("🔄 New Chat")

    # Event bindings
    submit_btn.click(user_submit, [user_input, chat_display], [user_input, chat_display])
    user_input.submit(user_submit, [user_input, chat_display], [user_input, chat_display])
    reset_btn.click(reset_history, outputs=[chat_display, user_input])

demo.launch()

* Running on local URL:  http://127.0.0.1:7866
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://aea9270aba35999204.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - The type of location they should end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "category": "...",
          "location_accident": "...",
          "description": "...",
          "type_of_final_location": "...",
          
        }
        Note: add more rows in the json file as you judge you get more and more

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - The type of location they should end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "category": "...",
          "location_accident": "...",
          "description": "...",
          "type_of_final_location": "...",
          
        }
        Note: add more rows in the json file as you judge you get more and more

In [120]:
import gradio as gr

# 🧠 Your existing function assumed
def gradio_chat(message, history):
    response, _ = insurance_chat(message)  # from your code
    return response

# 🌐 Avatar image links
USER_AVATAR = "https://i.imgur.com/VXhSkSX.png"  # Replace with your own hosted image
BOT_AVATAR = "https://i.imgur.com/Z8bnZ8x.png"

# 🧱 Format messages with avatar + speech bubble style
def format_message(message, is_user):
    avatar = USER_AVATAR if is_user else BOT_AVATAR
    align = "left" if is_user else "right"
    bg_color = "#dcf8c6" if is_user else "#ececec"
    return f"""
    <div style="display: flex; justify-content: {align}; margin: 5px 0;">
      <img src="{avatar}" width="30" style="border-radius: 50%; margin-right: 10px;">
      <div style="background: {bg_color}; padding: 8px 12px; border-radius: 10px; max-width: 70%; word-wrap: break-word;">
        {message}
      </div>
    </div>
    """

# 🧹 Reset history backend
def reset_history():
    conversation_histories["default"] = []
    return [], ""

# 🔁 Submit logic
def user_submit(user_message, chat_display):
    response = gradio_chat(user_message, chat_display)
    conversation_histories["default"].append(user_message)
    conversation_histories["default"].append(response)

    user_msg_html = format_message(user_message, is_user=True)
    bot_msg_html = format_message(response, is_user=False)

    updated_display = chat_display + [user_msg_html, bot_msg_html]
    return "", updated_display

# 🎛️ Gradio App
with gr.Blocks(title="Insurance Chat Pro") as demo:
    gr.Markdown("## 💬 Insurance Agent Chatbot\nTalk to an AI-powered assistant about your insurance needs.")

    chat_display = gr.HTML()
    user_input = gr.Textbox(placeholder="Type your message here...", show_label=False)

    with gr.Row():
        submit_btn = gr.Button("📤 Send", variant="primary")
        reset_btn = gr.Button("🔄 New Chat")

    # Button logic
    submit_btn.click(user_submit, [user_input, chat_display], [user_input, chat_display])
    user_input.submit(user_submit, [user_input, chat_display], [user_input, chat_display])
    reset_btn.click(reset_history, outputs=[chat_display, user_input])

demo.launch()

* Running on local URL:  http://127.0.0.1:7862
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://642d2ff1d77f37acbd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - The type of location they should end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "category": "...",
          "location_accident": "...",
          "description": "...",
          "type_of_final_location": "...",
          
        }
        Note: add more rows in the json file as you judge you get more and more

In [118]:
import gradio as gr
# Gradio interface wrapper
def gradio_chat(prompt):
    response, _ = insurance_chat(prompt)  # ignoring extracted JSON
    return response

# Run the UI
gr.Interface(
    fn=gradio_chat,
    inputs=gr.Textbox(label="🧍 User", placeholder="Ask me about your insurance situation..."),
    outputs=gr.Textbox(label="🤖 Bot"),
    title="Insurance AI Chatbot",
    description="Chat with an assistant trained to handle insurance scenarios."
).launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://2ffd7fea6d7939675a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - The type of location they should end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "category": "...",
          "location_accident": "...",
          "description": "...",
          "type_of_final_location": "...",
          
        }
        Note: add more rows in the json file as you judge you get more and more

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⚠️ JSON parsing failed. Raw output:
 Extract the following information from the client message history:
        
        - Full Name
        - Category of Message (as mentioned above)
        - Location of Accident/Malfunction
        - What they want to do (call road assistance to come fix the car, call the tow truck to take their car away,
        call accident care to note the accident who was at fault etc.)
        - The type of location they should end up (service center, home etc.)
        - The exact location of the above location point
    
        If one of the pieces of info is not available just put the word None on it and return the JSON normally
        
        Respond only with a valid JSON object:
        {
          "name": "...",
          "category": "...",
          "location_accident": "...",
          "description": "...",
          "type_of_final_location": "...",
          
        }
        Note: add more rows in the json file as you judge you get more and more

In [104]:
from datasets import Dataset, DatasetDict
import pandas as pd

# Convert your conversations to DataFrame
df = pd.DataFrame(training_examples)

# Create Hugging Face dataset
dataset = Dataset.from_pandas(df)

# Split into train/test if needed (80/20 split)
dataset = dataset.train_test_split(test_size=0.2)

In [105]:
def format_conversation(example):
    # Apply the chat template
    example["text"] = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return example

# Format all examples
dataset = dataset.map(format_conversation)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=2048)

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [111]:
# Load quantized model
from transformers import BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # adapt this based on the model architecture
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

alt_model = get_peft_model(model, lora_config)

In [112]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./insurance_llama_finetuned",
    eval_strategy="steps",
    eval_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    report_to="tensorboard",  # Enables progress bar
    logging_strategy="steps",  # Makes logging show up in the bar
    disable_tqdm=False,  # Ensures progress bar is visible
    save_steps=1000,
    fp16=True,
    gradient_accumulation_steps=4,
)

trainer = Trainer(
    model=alt_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [98]:
# from transformers import DataCollatorForLanguageModeling

# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer,
#     mlm=False,
# )

# # Update trainer with collator
# trainer.data_collator = data_collator
trainer.data_collator = None

In [113]:
trainer.train()

# Save the trained model
trainer.save_model("./insurance_llama_finetuned")
tokenizer.save_pretrained("./insurance_llama_finetuned")

ValueError: expected sequence of length 303 at dim 1 (got 559)

In [93]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
import torch

# 1. Initialize tokenizer with padding
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Set padding token (using EOS token if no dedicated pad token exists)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # or tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# 2. Prepare data collator with proper padding
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # We're doing causal language modeling
    pad_to_multiple_of=8  # Helps with GPU efficiency
)

# 3. Verify your dataset is properly tokenized
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=2048,
        padding=False  # We'll handle padding in the collator
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["messages", "category"])
tokenized_dataset.set_format("torch")

# 4. Training arguments with progress tracking
training_args = TrainingArguments(
    output_dir="./insurance_llama_finetuned",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=1000,
    fp16=True,
    gradient_accumulation_steps=4,
    report_to="tensorboard",  # For progress visualization
    disable_tqdm=False,  # Show progress bar
    load_best_model_at_end=True,
    optim="adamw_torch",  # Recommended optimizer
    logging_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    dataloader_pin_memory=True,  # Improves speed
    dataloader_num_workers=2,  # If you have multiple CPU cores
)

# 5. Initialize model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 6. Create trainer with proper collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# 7. Start training with progress tracking
print("Starting training...")
trainer.train()

# 8. Save the trained model
trainer.save_model("./insurance_llama_finetuned")
tokenizer.save_pretrained("./insurance_llama_finetuned")
print("Training complete and model saved!")

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
403 Client Error. (Request ID: Root=1-6831ed5e-33f4045f03bb50685b712b1f;7826fc47-2dcb-4d64-ab1e-ea2aaa30437d)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct to ask for access.